# 0X Notebook-Template

In [1]:
import sys

sys.path.append("../../")

import pandas as pd
import connection as connsettings

from pyspark.sql import functions as f

from etl.etl import HadoopStdETL
from etl.clients import HiveClient, HDFileSystemClient
from etl.datamodels import HostDataClass, TableDataClass
from etl.enums import eHdfsFileType
from etl.connectors import CsvConnector

In [2]:
conn = CsvConnector(path="./database/datev.dbo.client.csv", sep="|")
conn

CsvConnector(
	'path=./database/datev.dbo.client.csv',
	'sep=|'
)

In [3]:
for i, batch in conn.iterbatches(batchsize=3):
    display(batch)

,id,name,address,change_field
0,57,Omicron Systems GmbH,"Musterstraße 57, 10163 Berlin",cf
1,19,Epic Solutions SE,"Musterstraße 19, 10125 Berlin",cf
2,66,Quantum Consulting SE,"Musterstraße 66, 10172 Berlin",cf


,id,name,address,change_field
3,30,Harmony Services GmbH,"Musterstraße 30, 10136 Berlin",cf
4,17,Echo Solutions SE,"Musterstraße 17, 10123 Berlin",cf
5,50,Mu Enterprises AG,"Musterstraße 50, 10156 Berlin",cf


,id,name,address,change_field
6,49,Momentum Innovations SE,"Musterstraße 49, 10155 Berlin",cf
7,92,Xcel Ventures AG,"Musterstraße 92, 10198 Berlin",cf
8,91,Wavelength Ventures AG,"Musterstraße 91, 10197 Berlin",cf


,id,name,address,change_field
9,3,Apex Corporation GmbH,"Musterstraße 3, 10109 Berlin",cf uc: 1


In [4]:
hive_host = HostDataClass(host=connsettings.HIVE_HOST, port=connsettings.HIVE_PORT)
hdfs_host = HostDataClass(host=connsettings.HDFS_HOST, port=connsettings.HDFS_PORT)
hdfs_client = HDFileSystemClient(hdfs_host=hdfs_host, hdfs_username="enricogoerlitz")
hive_client = HiveClient(host=hive_host, thrift_port=connsettings.HIVE_THRIFT_PORT)

In [22]:
hive_host = HostDataClass(host=connsettings.HIVE_HOST, port=connsettings.HIVE_PORT)
hdfs_host = HostDataClass(host=connsettings.HDFS_HOST, port=connsettings.HDFS_PORT)
hdfs_client = HDFileSystemClient(hdfs_host=hdfs_host, hdfs_username="enricogoerlitz")
hive_client = HiveClient(host=hive_host, thrift_port=connsettings.HIVE_THRIFT_PORT)

conn = CsvConnector(path="./database/datev.dbo.client.csv", sep="|")

etl = HadoopStdETL(
    conn=conn,
    hdfs_client=hdfs_client,
    hive_client=hive_client,
    tmp_path="/edw/hive/tmp/",
    bck_path="/edw/hive/bck_psa/",
    dist_path="/edw/hive/psa/",
    table=TableDataClass(
        database="datev",
        schema="dbo",
        table_name="client",
        pk=["id"]
    )
)

etl.start(batchsize=3)

/edw/hive/tmp/datev/dbo/client/BATCH0.parquet
/edw/hive/tmp/datev/dbo/client/BATCH1.parquet
/edw/hive/tmp/datev/dbo/client/BATCH2.parquet
/edw/hive/tmp/datev/dbo/client/BATCH3.parquet
UPDATE /edw/hive/psa/datev/dbo/client/client


NotImplementedError: 

In [6]:
tmp_table = TableDataClass("tmp_client", "datev")
table = TableDataClass("client", "datev")

tmp_table, table

(TableDataClass(table_name='tmp_client', database='datev', schema=None, pk=None),
 TableDataClass(table_name='client', database='datev', schema=None, pk=None))

In [7]:
hive_client.read_table(table).head()

,client.guid,client.pk,client.id,client.name,client.address,client.change_field,client.row_is_current,client.row_valid_from,client.row_valid_to,client.row_filepath
0,064bcad7-6974-41c7-97a9-fa8e2405a7f1,57,57,Omicron Systems GmbH,"Musterstraße 57, 10163 Berlin",cf,1,2024-01-03 20:35:21,2262-04-11 23:47:16,/edw/hive/tmp/datev/dbo/client/BATCH0.parquet
1,b8e09f28-7762-4c21-8b26-779b4b9a140a,19,19,Epic Solutions SE,"Musterstraße 19, 10125 Berlin",cf,1,2024-01-03 20:35:21,2262-04-11 23:47:16,/edw/hive/tmp/datev/dbo/client/BATCH0.parquet
2,76f66ef0-76e7-4019-952e-adc98d63b4a4,66,66,Quantum Consulting SE,"Musterstraße 66, 10172 Berlin",cf,1,2024-01-03 20:35:21,2262-04-11 23:47:16,/edw/hive/tmp/datev/dbo/client/BATCH0.parquet
3,24fb098f-b139-4186-9375-2c51cd0d1c32,30,30,Harmony Services GmbH,"Musterstraße 30, 10136 Berlin",cf,1,2024-01-03 20:35:21,2262-04-11 23:47:16,/edw/hive/tmp/datev/dbo/client/BATCH1.parquet
4,8fff39f0-b6ed-4124-b26c-23501782d585,17,17,Echo Solutions SE,"Musterstraße 17, 10123 Berlin",cf,1,2024-01-03 20:35:21,2262-04-11 23:47:16,/edw/hive/tmp/datev/dbo/client/BATCH1.parquet


In [8]:
hive_client.read_table(tmp_table).head()

,tmp_client.guid,tmp_client.pk,tmp_client.id,tmp_client.name,tmp_client.address,tmp_client.change_field,tmp_client.row_is_current,tmp_client.row_valid_from,tmp_client.row_valid_to,tmp_client.row_filepath
0,6526ad75-eafe-4401-b764-f3f2588aa03d,57,57,Omicron Systems GmbH,"Musterstraße 57, 10163 Berlin",cf,1,2024-01-03 20:35:28,2262-04-11 23:47:16,/edw/hive/tmp/datev/dbo/client/BATCH0.parquet
1,ba48bac7-6db3-4e0f-8f56-e27c51363fff,19,19,Epic Solutions SE,"Musterstraße 19, 10125 Berlin",cf,1,2024-01-03 20:35:28,2262-04-11 23:47:16,/edw/hive/tmp/datev/dbo/client/BATCH0.parquet
2,8e0ba3f3-1269-4d48-981b-c5eb378e60b7,66,66,Quantum Consulting SE,"Musterstraße 66, 10172 Berlin",cf,1,2024-01-03 20:35:28,2262-04-11 23:47:16,/edw/hive/tmp/datev/dbo/client/BATCH0.parquet
3,0a0a9c40-5579-4aa7-a9d8-36b8c53513cd,30,30,Harmony Services GmbH,"Musterstraße 30, 10136 Berlin",cf,1,2024-01-03 20:35:28,2262-04-11 23:47:16,/edw/hive/tmp/datev/dbo/client/BATCH1.parquet
4,9cb7ae45-972c-41d5-973b-5bd58cc10155,17,17,Echo Solutions SE,"Musterstraße 17, 10123 Berlin",cf,1,2024-01-03 20:35:28,2262-04-11 23:47:16,/edw/hive/tmp/datev/dbo/client/BATCH1.parquet


In [ ]:
for batch in hive_client.read_table_iterbatches(tmp_table, batchsize=3):
    display(batch)

In [9]:
spark = hive_client.create_spark_session()
df_tmp_client = spark.sql(f"SELECT * FROM datev.tmp_client")
df_client = spark.sql(f"SELECT * FROM datev.client WHERE ROW_IS_CURRENT = 1")
df_tmp_client.show()
df_client.show()

24/01/03 20:37:19 WARN Utils: Your hostname, MacBook-Air-von-Enrico.local resolves to a loopback address: 127.0.0.1; using 192.168.0.6 instead (on interface en0)
24/01/03 20:37:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/03 20:37:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+--------------------+---+---+--------------------+--------------------+------------+--------------+-------------------+-------------------+--------------------+
|                guid| pk| id|                name|             address|change_field|row_is_current|     row_valid_from|       row_valid_to|        row_filepath|
+--------------------+---+---+--------------------+--------------------+------------+--------------+-------------------+-------------------+--------------------+
|79e39731-c115-43d...| 49| 49|Momentum Innovati...|Musterstraße 49, ...|          cf|             1|2024-01-03 20:35:28|2262-04-11 23:47:16|/edw/hive/tmp/dat...|
|ffce23f1-0dde-48e...| 92| 92|    Xcel Ventures AG|Musterstraße 92, ...|          cf|             1|2024-01-03 20:35:28|2262-04-11 23:47:16|/edw/hive/tmp/dat...|
|0fab56be-e0f5-464...| 91| 91|Wavelength Ventur...|Musterstraße 91, ...|          cf|             1|2024-01-03 20:35:28|2262-04-11 23:47:16|/edw/hive/tmp/dat...|
|6526ad75-eafe-440...| 57| 5

In [10]:
df_tmp_client.join(df_client, on="pk").show()

+---+--------------------+---+--------------------+--------------------+------------+--------------+-------------------+-------------------+--------------------+--------------------+---+--------------------+--------------------+------------+--------------+-------------------+-------------------+--------------------+
| pk|                guid| id|                name|             address|change_field|row_is_current|     row_valid_from|       row_valid_to|        row_filepath|                guid| id|                name|             address|change_field|row_is_current|     row_valid_from|       row_valid_to|        row_filepath|
+---+--------------------+---+--------------------+--------------------+------------+--------------+-------------------+-------------------+--------------------+--------------------+---+--------------------+--------------------+------------+--------------+-------------------+-------------------+--------------------+
| 49|79e39731-c115-43d...| 49|Momentum Innovat

In [14]:
from pyspark.sql import functions as f

# update dummy
df_tmp_client = df_tmp_client.withColumn("name", f.when(f.col("pk") == 91, "here_name_changed").otherwise(f.col("name")))
df_tmp_client = df_tmp_client.withColumn("name", f.when(f.col("pk") == 92, "here_name_changed").otherwise(f.col("name")))
df_tmp_client = df_tmp_client.withColumn("name", f.when(f.col("pk") == 57, "here_name_changed").otherwise(f.col("name")))

# delete dummy
df_tmp_client = df_tmp_client.filter(f.col("pk") != 30)
df_tmp_client = df_tmp_client.filter(f.col("pk") != 66)
df_tmp_client = df_tmp_client.filter(f.col("pk") != 17)

df_client = df_client.withColumn("CHANGE_KEY", f.concat_ws("_", *["name", "change_field"]))
df_tmp_client_ = df_tmp_client.withColumn("CHANGE_KEY_TMP", f.concat_ws("_", *["name", "change_field"]))

df_tmp_client_ = df_tmp_client_.withColumnRenamed("guid", "GUID_TMP")

df_tmp_client_ = df_tmp_client_.select(*["pk", "GUID_TMP", "CHANGE_KEY_TMP"])
df_joined = df_client.join(df_tmp_client_, on="pk", how="left")


df_updated_rows = df_joined.filter(f.col("CHANGE_KEY") != f.col("CHANGE_KEY_TMP"))
df_deleted_rows = df_joined.filter(f.col("CHANGE_KEY_TMP").isNull())

#df_tmp_client.show()
# df_joined.show()
df_updated_rows_pd = pd.DataFrame(
    df_updated_rows.collect(),
    columns=df_updated_rows.columns
)
display(df_updated_rows_pd)

df_deleted_rows_pd = pd.DataFrame(
    df_deleted_rows.collect(),
    columns=df_deleted_rows.columns
)
display(df_deleted_rows_pd)

# READ FILE
# TRANSFORM IT
#   - change and add old dataset ==> need GUID & FILEPATH from old dataset
# WRITE IT BACK (override!)
# ==> MIT PANDAS!

# print("CHANGED COLUMNS:")
# df_updated_rows.show()
# df_deleted_rows.show()
hdfs_client.read_parquet("/edw/hive/tmp/datev/dbo/client/BATCH2.parquet")


# UND --> wenn filename doppelt! nur einmal öffnen und verändern!

# UPDATE UND DELETE FASSE ICH ABER AUCH NUR EINMAL AN!

,pk,guid,id,name,address,change_field,row_is_current,row_valid_from,row_valid_to,row_filepath,CHANGE_KEY,GUID_TMP,CHANGE_KEY_TMP
0,92,7d059bc0-1dc5-4698-a4f8-21c22522a82f,92,Xcel Ventures AG,"Musterstraße 92, 10198 Berlin",cf,1,2024-01-03 20:35:21,2262-04-11 23:47:16,/edw/hive/tmp/datev/dbo/client/BATCH2.parquet,Xcel Ventures AG_cf,ffce23f1-0dde-48e2-b77a-89a79ce5095b,here_name_changed_cf
1,91,aa9d764b-b701-4681-bd38-da45fa12ca2c,91,Wavelength Ventures AG,"Musterstraße 91, 10197 Berlin",cf,1,2024-01-03 20:35:21,2262-04-11 23:47:16,/edw/hive/tmp/datev/dbo/client/BATCH2.parquet,Wavelength Ventures AG_cf,0fab56be-e0f5-4646-8813-83a89cb04471,here_name_changed_cf
2,57,064bcad7-6974-41c7-97a9-fa8e2405a7f1,57,Omicron Systems GmbH,"Musterstraße 57, 10163 Berlin",cf,1,2024-01-03 20:35:21,2262-04-11 23:47:16,/edw/hive/tmp/datev/dbo/client/BATCH0.parquet,Omicron Systems GmbH_cf,6526ad75-eafe-4401-b764-f3f2588aa03d,here_name_changed_cf


,pk,guid,id,name,address,change_field,row_is_current,row_valid_from,row_valid_to,row_filepath,CHANGE_KEY,GUID_TMP,CHANGE_KEY_TMP
0,66,76f66ef0-76e7-4019-952e-adc98d63b4a4,66,Quantum Consulting SE,"Musterstraße 66, 10172 Berlin",cf,1,2024-01-03 20:35:21,2262-04-11 23:47:16,/edw/hive/tmp/datev/dbo/client/BATCH0.parquet,Quantum Consulting SE_cf,None,None
1,30,24fb098f-b139-4186-9375-2c51cd0d1c32,30,Harmony Services GmbH,"Musterstraße 30, 10136 Berlin",cf,1,2024-01-03 20:35:21,2262-04-11 23:47:16,/edw/hive/tmp/datev/dbo/client/BATCH1.parquet,Harmony Services GmbH_cf,None,None
2,17,8fff39f0-b6ed-4124-b26c-23501782d585,17,Echo Solutions SE,"Musterstraße 17, 10123 Berlin",cf,1,2024-01-03 20:35:21,2262-04-11 23:47:16,/edw/hive/tmp/datev/dbo/client/BATCH1.parquet,Echo Solutions SE_cf,None,None


,id,name,address,change_field,GUID,PK,ROW_VALID_FROM,ROW_VALID_TO,ROW_IS_CURRENT,ROW_FILEPATH
0,49,Momentum Innovations SE,"Musterstraße 49, 10155 Berlin",cf,79e39731-c115-43d2-9b09-44f09920f689,49,2024-01-03 20:35:28,2262-04-11 23:47:16,1,/edw/hive/tmp/datev/dbo/client/BATCH2.parquet
1,92,Xcel Ventures AG,"Musterstraße 92, 10198 Berlin",cf,ffce23f1-0dde-48e2-b77a-89a79ce5095b,92,2024-01-03 20:35:28,2262-04-11 23:47:16,1,/edw/hive/tmp/datev/dbo/client/BATCH2.parquet
2,91,Wavelength Ventures AG,"Musterstraße 91, 10197 Berlin",cf,0fab56be-e0f5-4646-8813-83a89cb04471,91,2024-01-03 20:35:28,2262-04-11 23:47:16,1,/edw/hive/tmp/datev/dbo/client/BATCH2.parquet


In [12]:
df_updated_rows_pd = pd.DataFrame(df_updated_rows.collect(), columns=df_updated_rows.columns)
df_updated_rows_pd

,pk,guid,id,name,address,change_field,row_is_current,row_valid_from,row_valid_to,row_filepath,CHANGE_KEY,GUID_TMP,CHANGE_KEY_TMP
0,91,b77314dd-59fa-4d73-ae13-97d1339dc000,91,Wavelength Ventures AG,"Musterstraße 91, 10197 Berlin",cf,1,2024-01-03 19:52:19,2262-04-11 23:47:16,/edw/hive/tmp/datev/dbo/client/BATCH8.parquet,Wavelength Ventures AG_cf,3dd0e68b-6347-45b8-9864-ccc0c6467972,here_name_changed_cf


In [13]:
hdfs_client.client.status("/edw/hive/tmp/")

{'accessTime': 0,
 'blockSize': 0,
 'childrenNum': 1,
 'fileId': 40344,
 'group': 'supergroup',
 'length': 0,
 'modificationTime': 1704281132017,
 'owner': 'enricogoerlitz',
 'pathSuffix': '',
 'permission': '755',
 'replication': 0,
 'storagePolicy': 0,
 'type': 'DIRECTORY'}

In [21]:
parquet_file_path = '/edw/hive/tmp/datev/dbo/client/BATCH8.parquet'

hdfs_client.read_parquet(parquet_file_path)

,id,name,address,change_field,GUID,PK,ROW_VALID_FROM,ROW_VALID_TO,ROW_IS_CURRENT,ROW_FILEPATH
0,91,Wavelength Ventures AG,"Musterstraße 91, 10197 Berlin",cf,3dd0e68b-6347-45b8-9864-ccc0c6467972,91,2024-01-03 19:52:25,2262-04-11 23:47:16,1,/edw/hive/tmp/datev/dbo/client/BATCH8.parquet


In [19]:
import pyarrow.parquet as pq
from io import BytesIO

parquet_file_path = '/edw/hive/tmp/datev/dbo/client/BATCH8.parquet'
with hdfs_client.client.read(parquet_file_path) as f:
    display(pd.read_parquet(BytesIO(f.read())))
    # print(f.read())
    #print(f.read())

,id,name,address,change_field,GUID,PK,ROW_VALID_FROM,ROW_VALID_TO,ROW_IS_CURRENT,ROW_FILEPATH
0,91,Wavelength Ventures AG,"Musterstraße 91, 10197 Berlin",cf,3dd0e68b-6347-45b8-9864-ccc0c6467972,91,2024-01-03 19:52:25,2262-04-11 23:47:16,1,/edw/hive/tmp/datev/dbo/client/BATCH8.parquet


In [20]:
import pandas as pd
from pyarrow.hdfs import connect

# HDFS connection parameters
hdfs_host = connsettings.HDFS_HOST
hdfs_port = connsettings.HDFS_PORT
hdfs_user = 'enricogoerlitz'

# Path to the Parquet file on HDFS


# Create an insecure HDFS connection
hdfs = connect(host=hdfs_host, port=hdfs_port, user=hdfs_user)
hdfs.read_parquet(parquet_file_path)
# Read the Parquet file into a PyArrow Table
# table = pq.read_table(hdfs.open_input_file(parquet_file_path))

# # Convert the PyArrow Table to a Pandas DataFrame
# df = table.to_pandas()

# # Display the Pandas DataFrame
# print(df)


/var/folders/5p/1ptjghjd2zd_l124jpvx3sv00000gn/T/ipykernel_7962/179224570.py:13: FutureWarning: pyarrow.hdfs.connect is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  hdfs = connect(host=hdfs_host, port=hdfs_port, user=hdfs_user)


FileNotFoundError: [Errno 2] No such file or directory: 'hadoop'